# Requêtes Openstreetmap

 spécifique à l'API d'OSM pour récupérer nos données au format json.

Nous allons effectuer des requêtes [Openstreetmap](https://www.openstreetmap.org/) (OSM). Le [wiki](https://wiki.openstreetmap.org/wiki/Main_Page) est une ressources quasi-indispensable pour bien gérer les données issues d'OSM (c'est parfois un peu obscur de comment tout ça fonctionne).

## Données géographiques OSM


Openstreetmap connait 3 types de données : 

- les noeuds : `node` : une coordonnée
- les chemins :  `way` : une liste ordonnée de nœuds (une route ou un contours)
- les relations : `relation` : une liste ordonnée de nœuds, de chemains ou d'autres relations.

Certains objets pouvant être décrits comme des chemins ou des relations selon la personne qui a référencé l'objet :

- l'ecm est un chemin : <https://www.openstreetmap.org/way/527054532>
- le chateau de versailles une relation : <https://www.openstreetmap.org/relation/1149002>
- Marseille est une relation également : <https://www.openstreetmap.org/relation/76469>

## Requêtes

On peut effectuer des requêtes OSM directement sur le site <https://overpass-turbo.eu/> ou en utisant une API avec le site <http://overpass-api.de>.

> Il est nécessaire de bien la documentation : <https://dev.overpass-api.de/overpass-doc/fr/index.html>

1. Nous allons commencer par nous familiariser avec les requêtes directement sur le site
2. Forger nos requête avec l'API pour les utiliser dans nos représentations graphiques

### Appli web

1. Rendez vous sur le site <http://overpass-turbo.eu/>
2. Centrez la carte sur l'ecm. Gardez la carte petite pour ne prendre que les bâtiments de l'ecm
3. tapez la requête ci-après dans l'éditeur de requête et exécutez là.

```
[out:json][timeout:25];
(  node({{bbox}});   
   way({{bbox}});   
   relation({{bbox}}); 
); 

out body;
```

> Si vous prenez une carte trop grande, le site vous averira que vous avez trop de données à télécharger.

Sur la carte seuls les `node` sont représentés (en utilisant leurs données `lat` et `lon`). En regardant les données vous verrez également, au format json, les `way` et les `relations` qui n'ont pas de coordonnées par défaut.

#### selection bbox

Notre sélection se fait actuellement via la carte un utilisant le paramètre `{{bbox}}`.

On peut également choisir sa propre bbox, par exemple : 

```
(43.33863,5.4336,43.34709,5.44304)
```

Qui correspond aux coordonnées (y1,x1, y2, x2) en latitude/longitude de l'ecm.

On peut alors faire la requête suivante, indépendante de la carte du site :

```
[out:json][timeout:25];
(  node(43.33863,5.4336,43.34709,5.44304);   
   way(43.33863,5.4336,43.34709,5.44304);   
   relation(43.33863,5.4336,43.34709,5.44304); 
); 

out body;
```

> Vous pouvez trouver vos propres bbox avec le site <https://norbertrenner.de/osm/bbox.html>

#### Sélection par area

Enfin, opn peut utiliser les bornes géographique d'un `way` ou d'une `relation` pour récupérer tout ce qui'il y a à l'intérieur. Par exemple dans l'em, qui est un `way` :

<https://www.openstreetmap.org/way/527054532>

On peut utiliser le code suivant, qui transforme le way en area puis fait la recherche :

```
[out:json][timeout:25];

way(527054532) -> .b;
.b map_to_area -> .a;

(  	node(area.a);   
	way(area.a);   
	relation(area.a); 
); 
out body;
```

> Lisez la doc pour voir les différentes possibilités :
> - <https://osmlab.github.io/learnoverpass/en/docs/filters/area/>
> - <https://wiki.cartocite.fr/doku.php?id=tutoverpass:jour_9_l_instruction_map_to_area>
> - <https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL#Map_way/relation_to_area_(map_to_area)>

## Compter les pizzeria

Notre but sera ici de compter le nombre pizzéria marseillaises par arrondissement pour les représenter sur une carte chloroplète.

Pour délimiter Marseille, on utilisera la relation : <https://www.openstreetmap.org/relation/76469>

On ne peut pas chercher tout les éléments de Marseille, il faut restreindre aux pizzeria. Pour cela :

- on sélectionne des tags dans la recherche : <https://osmlab.github.io/learnoverpass/en/docs/filters/tag/>
- les possiboités sont énorme, voir : <https://wiki.openstreetmap.org/wiki/FR:%C3%89l%C3%A9ments_cartographiques>

### Sur le site

Sur le sire cela revient à utiliser la requête :

```
[out:json][timeout:25];

rel(76469) -> .b;
.b map_to_area -> .a;

(  	node["cuisine"~"pizza"](area.a);   
	way["cuisine"~"pizza"](area.a);   
	relation["cuisine"~"pizza"](area.a); 
); 
out body;
```

Remarquer qu'il faut que : le champ cuisine soit renseigné et qu'il contienne `"pizza"`. Au final cela ne fait pas toutes les pizzerias de Marseille (et de loin) et il manque également les camions-pizza.

### Récupérer les données

Pour l'instant, les données sont récupérées sur un site et pas dasn le notebook. On peut utiliser le module [requests](https://requests-fr.readthedocs.io/en/latest/) :

In [ ]:
import requests
import json

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25];

rel(76469) -> .b;
.b map_to_area -> .a;

(  	node["cuisine"~"pizza"](area.a);   
	way["cuisine"~"pizza"](area.a);   
	relation["cuisine"~"pizza"](area.a); 
); 
out body;
"""

réponse = requests.get(overpass_url, params={'data': overpass_query})

la réponse est un [objet complexe](https://requests.readthedocs.io/en/latest/user/quickstart/#response-content). Dans notre cas, elle contient un objet json qu'il nous faut convertir en dictgionnaire python :

In [ ]:
data = réponse.json()

In [ ]:
# data  # attention c'est gros

Les réponses sont placées dans une liste :

In [ ]:
data["elements"][:5]

Il y a deux des trois types possible :

In [ ]:
type_elements = set()
for e in data["elements"]:
    type_elements.add(e["type"])

print(type_elements)

In [ ]:
for e in data["elements"]:
    if e["type"] == "node":
        print(json.dumps(e, indent=2))
        break

In [ ]:
for e in data["elements"]:
    if e["type"] == "way":
        print(json.dumps(e, indent=2))
        break

On remarque que les éléments de type `way` n'ont pas de coordonnées. Ce qui va être problématique pour la suite lorsqu'il faudra les ranger par earrondissement. Il faut refaire la requête en ajoutant un paramètre à exporter, `center` :

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25];

rel(76469) -> .b;
.b map_to_area -> .a;

(  	node["cuisine"~"pizza"](area.a);   
	way["cuisine"~"pizza"](area.a);   
	relation["cuisine"~"pizza"](area.a); 
); 
out body center;
"""

pizzerias_json_raw = requests.get(overpass_url, params={'data': overpass_query}).json()["elements"]

In [ ]:
pizzerias_json_raw[:5]

Vérifions que l'on peut bien extraire les coordonnées de nos pizzérias :

In [ ]:
for e in pizzerias_json_raw:
    if e["type"] == "node":
        print(json.dumps(e, indent=2))
        break

In [ ]:
for e in pizzerias_json_raw:
    if e["type"] == "way":
        print(json.dumps(e, indent=2))
        break

On va créer nos données json on ne conservant que les coordonnées et le nom. Attention certaines pizzeria n'ont pas de nom :

In [ ]:
for e in pizzerias_json_raw:
    if "name" not in e["tags"]:
        print(json.dumps(e, indent=2))

In [ ]:
pizzerias_json = []
for e in pizzerias_json_raw:
    if e["type"] == "node":
        pizzerias_json.append({
            "lat": e["lat"],
            "lon": e["lon"],
            "nom": e["tags"].get("name", None)})
    else:
        pizzerias_json.append({
            "lat": e["center"]["lat"],
            "lon": e["center"]["lon"],
            "nom": e["tags"].get("name", None)})

In [ ]:
pizzerias_json[:5]

Avant de créer le geodataframe, commençpns par ranger nos données dans un dataframe :

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(pizzerias_json)

In [ ]:
df

On peut maintenant créer le GeoDataframe.

In [ ]:
import geopandas as gpd

In [ ]:
pizzerias = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)

In [ ]:
pizzerias

In [ ]:
pizzerias.crs

Il faut maintenant trouver les arrondissements de marseille et faire le comptage. On peut utiliser deux jeux de données :

- Les arrondissements disponibles depuis : <https://laprovence.carto.com/tables/arrondissements/public/map>
- les quartiers disponible depuis <https://www.data.gouv.fr/fr/datasets/quartiers-de-marseille-1/>

Prenons les quartiers de marseille

In [ ]:
quartiers = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6")

In [ ]:
quartiers

In [ ]:
quartiers.crs

In [ ]:
pizzeria_quartier = gpd.sjoin(pizzerias, quartiers, how="inner", predicate='intersects')

In [ ]:
pizzeria_quartier

In [ ]:
compte = (pizzeria_quartier
             .assign(nombre=1)
             .groupby(by="NOM_QUA", as_index=False)
             .sum(numeric_only=True)
        )
compte

In [ ]:
quartiers_compte = quartiers.merge(compte[['NOM_QUA', 'nombre']], how='inner', on='NOM_QUA')

In [ ]:
quartiers_compte

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx
import xyzservices.providers as xyz

On fait un peut de magie pour rajouter de la transparence à nos couleurs

In [ ]:
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap

In [ ]:
cmap = pl.cm.OrRd


my_cmap = cmap(list(range(cmap.N)))
my_cmap[:,-1] = [.5] * cmap.N

cmap = ListedColormap(my_cmap)

Puis on peut dessiner notre carte du nombre de pizzeria par quartier, en ajoutant les délimations de chaque quartier :

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.axis(False)

quartiers.boundary.plot(ax=ax)
quartiers_compte.plot(column='nombre', 
             legend=True,
             legend_kwds={'label': "Nombre de pizzeria", 'orientation': "horizontal"},
             cmap=cmap,
             ax=ax)
ctx.add_basemap(ax, crs="epsg:4326", source=xyz.GeoportailFrance.plan)

plt.show()